<a href="https://colab.research.google.com/github/mihika-17/DataScienceEcosystem/blob/main/app_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit pyngrok openpyxl


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.7 MB/s eta 0:00:00


In [19]:
%%writefile app.py
import streamlit as st
import pandas as pd
import re
import matplotlib.pyplot as plt

st.title("📊 Excel Sheet Consolidator")
st.write("Upload an Excel file with sheets named like 'March 2025'.")

uploaded_file = st.file_uploader("Upload Excel file", type=["xlsx", "xls"])

if uploaded_file is not None:
    xls = pd.ExcelFile(uploaded_file)
    month_year_pattern = re.compile(
        r"^(January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{4}$",
        re.IGNORECASE
    )

    dfs = []
    for sheet_name in xls.sheet_names:
        if month_year_pattern.match(sheet_name):
            df = pd.read_excel(uploaded_file, sheet_name=sheet_name)
            df["Sheet Name"] = sheet_name
            dfs.append(df)

    if dfs:
        consolidated_df = pd.concat(dfs, ignore_index=True)
        st.success(f"✅ Consolidated {len(dfs)} sheets.")
        st.dataframe(consolidated_df)

        if "Incident Type" in consolidated_df.columns:
            st.subheader("📌 Top Incident Types")
            incident_counts = consolidated_df["Incident Type"].value_counts().reset_index()
            incident_counts.columns = ["Incident Type", "Count"]

            st.dataframe(incident_counts)
            st.bar_chart(incident_counts.set_index("Incident Type"))
        else:
            st.warning("⚠️ Column 'Incident Type' not found in the data.")

        # === ⏳ Average Delay in Forwarding ===
        if "Incident Received by QA on" in consolidated_df.columns and "Date" in consolidated_df.columns:
            # Convert columns to datetime
            consolidated_df["Incident Received by QA on"] = pd.to_datetime(consolidated_df["Incident Received by QA on"], errors='coerce')
            consolidated_df["Date"] = pd.to_datetime(consolidated_df["Date"], errors='coerce')

            # Calculate delay in days
            consolidated_df["Forwarding Delay (Days)"] = (consolidated_df["Incident Received by QA on"] - consolidated_df["Date"]).dt.days

            # Calculate average delay
            avg_delay = consolidated_df["Forwarding Delay (Days)"].mean(skipna=True)

            st.subheader("⏳ Average Delay in Forwarding to QA")
            st.metric(label="Average Delay (Days)", value=round(avg_delay, 2))
        # === 📈 Histogram of delays ===
            st.subheader("📊 Distribution of Forwarding Delays")
            valid_delays = consolidated_df["Forwarding Delay (Days)"].dropna()

            if not valid_delays.empty:
                fig, ax = plt.subplots()
                ax.hist(valid_delays, bins=20, color='skyblue', edgecolor='black')
                ax.set_xlabel("Delay (Days)")
                ax.set_ylabel("Number of Incidents")
                ax.set_title("Distribution of Forwarding Delays")
                st.pyplot(fig)
            else:
                st.info("No valid forwarding delay data to plot.")
        else:
            st.warning("⚠️ Required columns 'Incident Received by QA on' and/or 'Date' not found.")
    else:
        st.error("No sheets match the 'Month Year' format.")



Overwriting app.py


In [16]:
from pyngrok import ngrok
ngrok.set_auth_token("3108EAOlFFWQYf5Vxr8kBQXdY8J_5WWoawwt1Q64W1AnsE2q5")

In [ ]:
public_url = ngrok.connect(8501)
print("Streamlit app is live at:", public_url)

!streamlit run app.py & sleep 5


Streamlit app is live at: NgrokTunnel: "https://9fb769ca200a.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.90.44.161:8501

2025-08-12 11:19:37.478 Serialization of dataframe to Arrow table was unsuccessful. Applying automatic fixes for column types to make the dataframe Arrow-compatible.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/streamlit/dataframe_util.py", line 821, in convert_pandas_df_to_arrow_bytes
    table = pa.Table.from_pandas(df)
            ^^^^^^^^^^^^^^^^^^^^^^^^
  File "pyarrow/table.pxi", line 4751, in pyarrow.lib.Table.from_pandas
  File "/usr/local/lib/python3.11/dist-packages/pyarrow/pandas_compat.py", line 625, in dataframe_to_arrays
    arrays = [convert_column(c, f)
             ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pyarrow/pandas_compat.py